### Вебинар 4. Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

#module_path = os.path.abspath(os.path.join(os.pardir))
#if module_path not in sys.path:
#    sys.path.append(module_path)

from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

print('Все загружено корректно!')

Все загружено корректно!


In [2]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]


data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


In [6]:
mr = MainRecommender(data = data_train)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(5)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [8]:
#result = result.loc[result['user_id'] != 1984]
#result = result.loc[result['user_id'] != 2259]

In [9]:
result['als'] = result['user_id'].apply(lambda x: mr.get_model_recommendations(x))

In [10]:
result['own'] = result['user_id'].apply(lambda x: mr.get_own_recommendations(x))
result.head(3)

,user_id,actual,als,own
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[995785, 1082185, 961554, 995242, 1005186]","[995242, 1082185, 1029743, 6534178, 1005186]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5569230, 1106523, 1044078, 951590, 1098066]","[1106523, 951590, 1082185, 6534178, 5569230]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[844179, 878996, 849843, 981760, 986912]","[1082185, 5569230, 6534178, 1029743, 1127831]"


In [11]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.2131:als
0.2879:own


/Users/aleksandrinavatkina/Desktop/GeekBrains/Рекомендательные системы/copies hw/src/metrics.py:22: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)


In [12]:
result['als_similar_items'] = result['user_id'].apply(lambda x: mr.get_similar_items_recommendation(x))
result.head()

,user_id,actual,als,own,als_similar_items
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[995785, 1082185, 961554, 995242, 1005186]","[995242, 1082185, 1029743, 6534178, 1005186]","[1022981, 819845, 862682, 883370, 1110843]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5569230, 1106523, 1044078, 951590, 1098066]","[1106523, 951590, 1082185, 6534178, 5569230]","[1021324, 993390, 1096794, 883932, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[844179, 878996, 849843, 981760, 986912]","[1082185, 5569230, 6534178, 1029743, 1127831]","[12301109, 869857, 889731, 9469110, 974156]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1127831, 1106523, 1082185, 1029743, 1044078]","[1082185, 1106523, 1029743, 1126899, 1133018]","[958569, 993390, 819845, 1055155, 1096794]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1127831, 840361, 1106523, 849843, 826249]","[6534178, 1082185, 1106523, 5569230, 1029743]","[958569, 983050, 993390, 889731, 1098066]"


In [13]:
result['als_similar_users'] = result['user_id'].apply(lambda x: mr.get_similar_users_recommendation(x))

In [14]:
for name_col in result.columns[2:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

0.2131:als
0.2879:own
0.0387:als_similar_items
0.1804:als_similar_users


/Users/aleksandrinavatkina/Desktop/GeekBrains/Рекомендательные системы/copies hw/src/metrics.py:22: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)
